In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras 

Using TensorFlow backend.


In [0]:
from keras.models import Sequential

from keras.initializers import he_normal

from keras.layers.normalization import BatchNormalization

from keras.layers import Dense

from keras.layers import Activation # linear, tanh, softmax
from keras.layers import ReLU, LeakyReLU, ELU, PReLU

from keras.layers import Dropout

from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [0]:
xtrain = np.load("xtrain.npy")
ytrain = np.load("ytrain.npy")

In [0]:
import timeit

def train_model(xtrain, ytrain, layers,
                activation,
                optimizer,
                batch_normalization=False, 
                dropout=False,
                epochs=1000,
                batch_size=500,
                patience=250):
  
    assert len(layers)>0

    start = timeit.default_timer()

    model = Sequential()

    model.add(Dense(layers[0], input_dim=xtrain.shape[1], init=he_normal(seed=None)))
    if(batch_normalization):
       model.add(BatchNormalization())
    model.add(activation)
    if(dropout):
       model.add(Dropout(0.5))

    for N in layers[1:]:
       model.add(Dense(N, init=he_normal(seed=None)))
       if(batch_normalization):
          model.add(BatchNormalization())
       model.add(activation)
       if(dropout):
          model.add(Dropout(0.5))

    model.add(Dense(ytrain.shape[1], init=he_normal(seed=None)))
    model.add(Activation('linear'))

    model.compile(loss='mean_squared_error', optimizer=optimizer)
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience)
    
    history = model.fit(xtrain, ytrain, 
                        validation_split=0.2, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        verbose=0, 
                        callbacks=[early_stopping])
    
    stop = timeit.default_timer()
      
    training_time = np.round((stop-start)/60 , 2)

    return model, history, training_time

In [0]:
_ = plt.figure(figsize=(30,30))
_ = plt.title("3 hidden layers with 128 neurons each")
_ = plt.ylabel('Loss')
_ = plt.xlabel('Epochs')

out_file = open('training_times.txt', 'w')

for ACT in [ReLU, LeakyReLU, ELU, PReLU]:
  for OPT in [SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam]:
    for DROP in [False, True]:
      for BATCH in [False, True]:
            name = ''
            if(BATCH):
              name += 'BATCH_'
            name += ACT.__name__
            if(DROP):
              name += '_DROP'
            name += '_'
            name += OPT.__name__
            
            model, history, training_time = train_model(xtrain=xtrain,
                                                        ytrain=ytrain,
                                                        layers=[128,128,128],
                                                        activation=ACT(),
                                                        optimizer=OPT(),
                                                        dropout=DROP,
                                                        batch_normalization=BATCH,
                                                        epochs=3000)
            
            model.save(name+'.h5')
            out_file.write(name+' ---> '+str(training_time)+' minutes\n')

            _ = plt.semilogy(history.history['loss'],label=name+' train')
            _ = plt.semilogy(history.history['val_loss'],label=name+' test')

out_file.close()
_ = plt.legend(loc='upper right')
_ = plt.savefig('losses_graphs.png')